## Evironment Setup

In [ ]:
!pip install -q transformers datasets sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 18.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 13.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 11.7 MB/s eta 0:00:00


In [ ]:
# !nvidia-smi

In [ ]:
# Importing the libraries needed
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import torch
import seaborn as sns
import pickle
import transformers
import json
import copy
from os import listdir
from PIL import Image
import pprint

from tqdm import tqdm
from tqdm.notebook import tqdm
from torch.utils.data import Dataset, DataLoader

from transformers import LxmertTokenizer, LxmertModel
# tokenizer = LxmertTokenizer.from_pretrained('unc-nlp/lxmert-base-uncased')

import torch.nn.functional as F
from transformers import AdamW, AutoModel, set_seed
import logging
logging.basicConfig(level=logging.ERROR)


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd /content/drive/MyDrive/00_torino_adnlp/

/content/drive/MyDrive/00_torino_adnlp


| FormNLU     | Images | Title | Section | Table-Key | Table-Value | Form-Key | Form-Value |
|-------------|--------|-------|---------|-----------|-------------|----------|------------|
| Train       | 464+71 (535) |  1068 |    1560 |      2669 |        2672 |     3708 |       3568 |
| Val         | 66+10 (76)  |   152 |     215 |       380 |         379 |      524 |        510 |
| Test        | 131+15 (146) |   292 |     418 |       730 |         730 |     1009 |        978 |
| Printed     |     50 |    98 |     148 |       250 |         249 |      346 |        332 |
| HandWritten |     50 |   100 |     150 |       249 |         226 |      348 |        315 |

      category_mapping = {
          1: 'title', 2: 'section', 3: 'other', 4: 'form_key', 5: 'form_value', 6: 'table_key', 7: 'table_value'}

In [ ]:
# open pkl files

with open('formnlu_features/formnlu_all_feat_lower_train.pkl', 'rb') as f:
    train_all_feat = pickle.load(f)

with open('formnlu_features/formnlu_all_feat_lower_val.pkl', 'rb') as f:
    val_all_feat = pickle.load(f)

with open('formnlu_features/formnlu_all_feat_lower_test.pkl', 'rb') as f:
    test_all_feat = pickle.load(f)

In [ ]:
print(train_all_feat.keys())
print(train_all_feat['01502814_page-0'].keys())
print(train_all_feat['01502814_page-0']['objects'][0].keys())
print(len(train_all_feat.keys()))

print(val_all_feat.keys())
print(val_all_feat['00910692_page-1'].keys())
print(val_all_feat['00910692_page-1']['objects'][0].keys())
print(len(val_all_feat.keys()))

print(test_all_feat.keys())
print(test_all_feat['00917958_page-0'].keys())
print(test_all_feat['00917958_page-0']['objects'][0].keys())
print(len(test_all_feat.keys()))

dict_keys(['01502814_page-0', '01311266_page-0', '01312096_page-0', '01314156_page-0', '01322021_page-0', '01327411_page-0', '01327879_page-0', '01330181_page-0', '01330743_page-0', '01331561_page-0', '01333748_page-0', '01334844_page-0', '01339499_page-0', '01342494_page-0', '01344731_page-0', '01346455_page-0', '01355023_page-0', '01359605_page-1', '01359608_page-1', '01368409_page-0', '01369341_page-0', '01370460_page-0', '01370855_page-0', '01372560_page-0', '01373528_page-0', '01376050_page-0', '01381847_page-0', '01382283_page-0', '01382666_page-0', '01383077_page-0', '01384450_page-0', '01385004_page-1', '01393056_page-0', '01395037_page-0', '01396709_page-0', '01397263_page-0', '01398740_page-0', '01398791_page-1', '01400324_page-0', '01403379_page-0', '01406011_page-0', '01407789_page-0', '01408127_page-1', '01408579_page-0', '01408937_page-1', '01413192_page-0', '01421331_page-0', '01421595_page-0', '01424210_page-0', '01424827_page-0', '01425017_page-0', '01432424_page-0', '

#### debugging code (train - empty data exist)

In [ ]:
train_sample = copy.deepcopy(train_all_feat)
print(train_sample.keys())
print(train_sample['01502814_page-0'].keys())
print(train_sample['01502814_page-0']['objects'][0].keys())
print(len(train_sample.keys()))

dict_keys(['01502814_page-0', '01311266_page-0', '01312096_page-0', '01314156_page-0', '01322021_page-0', '01327411_page-0', '01327879_page-0', '01330181_page-0', '01330743_page-0', '01331561_page-0', '01333748_page-0', '01334844_page-0', '01339499_page-0', '01342494_page-0', '01344731_page-0', '01346455_page-0', '01355023_page-0', '01359605_page-1', '01359608_page-1', '01368409_page-0', '01369341_page-0', '01370460_page-0', '01370855_page-0', '01372560_page-0', '01373528_page-0', '01376050_page-0', '01381847_page-0', '01382283_page-0', '01382666_page-0', '01383077_page-0', '01384450_page-0', '01385004_page-1', '01393056_page-0', '01395037_page-0', '01396709_page-0', '01397263_page-0', '01398740_page-0', '01398791_page-1', '01400324_page-0', '01403379_page-0', '01406011_page-0', '01407789_page-0', '01408127_page-1', '01408579_page-0', '01408937_page-1', '01413192_page-0', '01421331_page-0', '01421595_page-0', '01424210_page-0', '01424827_page-0', '01425017_page-0', '01432424_page-0', '

In [ ]:
# Iterate through each key in the 'train' dictionary
for key, data in train_sample.items():
    # Filter out objects with missing values in 'category_id' or any other key you want to check
    data['objects'] = {idx: obj for idx, obj in data.get('objects', {}).items() if all(obj.get(k) is not None for k in ['category_name', 'bert_cls', 'text'])}

In [ ]:
# original & new mapping
# category_mapping = {1: 'Title', 2: 'Section', 3: 'Other', 4: 'Form_key', 5: 'Form_value', 6: 'Table_key', 7: 'Table_value'}
# {'Title': 0, 'Other': 1, 'Section': 2, 'Form_key': 3, 'Table_key': 4, 'Table_value': 5, 'Form_value': 6}

label_dict = {}
label_id = 0

for doc in train_sample:
  objects = train_sample[doc]['objects']
  for i in objects:
    #print(objects[i]['category_name'])
     if objects[i]['category_name'] not in label_dict:
      label_dict[objects[i]['category_name']] = label_id
      label_id += 1

print(label_dict)

{'section': 0, 'other': 1, 'title': 2, 'form_key': 3, 'table_key': 4, 'form_value': 5, 'table_value': 6}


## Dataloader

In [ ]:
import math
import torch

# one dimensional feature embedding
def positionalencoding1d(d_model, feature_list):
    """
    :param d_model: dimension of the model
    :param feature_list: length of positions
    :return: length*d_model position matrix
    """
    if d_model % 2 != 0:
        raise ValueError("Cannot use sin/cos positional encoding with "
                         "odd dim (got dim={:d})".format(d_model))
    pe = torch.zeros(1, d_model)
    feats = torch.tensor(feature_list)
    div_term = torch.exp((torch.arange(0, d_model, 2, dtype=torch.float) *
                         -(math.log(10000.0) / d_model)))
    pe[:, 0::2] = torch.sin(feats.float() * div_term)
    pe[:, 1::2] = torch.cos(feats.float() * div_term)
    pe = np.array(pe.tolist())
    return pe

positional_encoding = []

for i in range(100):
    positional_encoding.append(positionalencoding1d(768, i)[0])
positional_encoding = np.array(positional_encoding)

#print(positional_encoding) # (100, 768)

In [ ]:
# https://github.com/uclanlp/visualbert/blob/master/visualbert/dataloaders/coco_dataset.py
# https://huggingface.co/docs/transformers/model_doc/visual_bert

# if it has coco format
# coco format [x_min, y_min, width, height]
# converted to [x_min, y_min, x_max, y_max]

def normalize_bbox(bbox, page_info):
  x1 = bbox[0]
  y1 = bbox[1]
  x2 = bbox[2]
  y2 = bbox[3]

  width = page_info['width']
  height = page_info['height']

  return [x1/width, y1/height, (x1+x2)/width, (y1+y2)/height]


### custom dataset

In [ ]:
# amended doc_info & page_info

class CustomDataset(Dataset):
    def __init__(self, doc_info, padding_len=100):
        self.tokenizer = tokenizer
        self.doc_info = doc_info
        self.positional_encoding = positional_encoding
        self.padding_len = padding_len

    def __len__(self):
        return len(self.doc_info)

    def __getitem__(self, idx):
        key = list(self.doc_info.keys())[idx]
        page_info = self.doc_info[key]
        #print("key", key)

        texts = []
        target_id = []
        boxes = []
        bert_cls = []
        for i in range(len(page_info["objects"])):
            texts.append(page_info["objects"][i]["text"])
            target_id.append(label_dict[page_info["objects"][i]["category_name"]])
            boxes.append(copy.deepcopy(page_info["objects"][i]["bbox"]))
            bert_cls.append(page_info["objects"][i]["bert_cls"])

        text = " ".join(texts)

        #print(texts)
        #print("target_id", target_id)
        #print(boxes)
        #print("len(bert_cls)", len(bert_cls))

        # Get Fine-Grained Level Information
        inputs = self.tokenizer.encode_plus(
            text,
            None,
            add_special_tokens=True,
            max_length=512,
            pad_to_max_length=True,
            return_token_type_ids=True
        )
        ids = inputs['input_ids']
        mask = inputs['attention_mask']
        token_type_ids = inputs["token_type_ids"]

        object_mask = []
        visual_feat = copy.deepcopy(page_info["visual_list"])
        norm_bbox = [normalize_bbox(bbox, page_info) for bbox in boxes]

        #print("len(visual_feat)", len(visual_feat))
        #print(visual_feat[0])
        #print(norm_bbox)

        if len(bert_cls) >= self.padding_len:
        #if len(visual_feat) >= self.padding_len:
          bert_cls = bert_cls[:self.padding_len]
          visual_feat = visual_feat[:self.padding_len]
          norm_bbox = norm_bbox[:self.padding_len]
          object_mask = [1]*self.padding_len
          #target_id = [-100]*self.padding_len
          target_id = target_id[:self.padding_len] # not [-100]
          #print("if_target_id", target_id)


        else:
          size = len(visual_feat)
          bert_cls.extend([[0.0]*768]*(self.padding_len-len(bert_cls)))
          visual_feat.extend([[0.0]*2048]*(self.padding_len-len(visual_feat)))
          norm_bbox.extend([[0.0]*4]*(self.padding_len-len(norm_bbox)))
          object_mask = [1]*size+[0.0]*(self.padding_len-size) # 1 for obj, 0 the rest
          #print("object_mask", object_mask)
          target_id.extend([-100]*(self.padding_len-len(target_id)))
          #print("else_target_id", target_id)

        return {
            'ids': torch.tensor(ids, dtype=torch.long),
            'mask': torch.tensor(mask, dtype=torch.float),
            'token_type_ids': torch.tensor(token_type_ids, dtype=torch.long),
            'bert_cls': torch.tensor(bert_cls, dtype=torch.float),
            'visual_feat': torch.tensor(visual_feat, dtype=torch.float),
            'target': torch.tensor(target_id, dtype=torch.float),
            'positional_encoding': torch.tensor(self.positional_encoding, dtype = torch.float),
            'object_mask':torch.tensor(object_mask, dtype=torch.float),
            'norm_bbox':torch.tensor(norm_bbox, dtype=torch.float)
        }


## Model Definition

In [ ]:
# uclanlp/visualbert-vqa-coco-pre

import torch.nn.functional as F
from transformers import AdamW, VisualBertModel, set_seed

class New_model(torch.nn.Module):
    def __init__(self):
        super(New_model, self).__init__()
        self.l1 = VisualBertModel.from_pretrained("uclanlp/visualbert-vqa-coco-pre", output_hidden_states=True)
        self.pre_classifier = torch.nn.Linear(768, 768)
        self.dropout = torch.nn.Dropout(0.1)
        self.classifier = torch.nn.Linear(768, 7) # labels 7

    # visual_token_type_ids (difference w/ LXMERT)
    def forward(self, ids, mask, token_type_ids, bert_cls, visual_feat, pos_emb, attention_mask, norm_bbox):
        visual_token_type_ids = torch.ones(visual_feat.shape[:-1], dtype=torch.long).to(device, dtype = torch.long)
        visual_attention_mask = attention_mask.to(device, dtype = torch.float)
        output_1 = self.l1(input_ids=ids,
                           attention_mask=mask,
                           token_type_ids=token_type_ids,
                           visual_embeds=visual_feat,
                           visual_token_type_ids=visual_token_type_ids,
                           visual_attention_mask=visual_attention_mask)

        hidden_state = output_1.hidden_states[-1]
        visual_feat = hidden_state[:, 512:,:]
        output = self.pre_classifier(visual_feat)
        output = torch.nn.Tanh()(output)
        output = self.dropout(output)
        output = self.classifier(output)
        #output = output.squeeze(2)

        return output

In [ ]:
# from transformers import LxmertTokenizer, LxmertModel
# tokenizer = LxmertTokenizer.from_pretrained('unc-nlp/lxmert-base-uncased')

from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased", truncation=True, do_lower_case=True)

model = New_model()
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

New_model(
  (l1): VisualBertModel(
    (embeddings): VisualBertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=1)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
      (visual_token_type_embeddings): Embedding(2, 768)
      (visual_position_embeddings): Embedding(512, 768)
      (visual_projection): Linear(in_features=2048, out_features=768, bias=True)
    )
    (encoder): VisualBertEncoder(
      (layer): ModuleList(
        (0-11): 12 x VisualBertLayer(
          (attention): VisualBertAttention(
            (self): VisualBertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropou

## Metrics

In [ ]:
!pip install -q seqeval evaluate

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 2.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 4.9 MB/s eta 0:00:00


In [ ]:
import evaluate
import torch
import numpy as np
from seqeval.metrics import classification_report

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

metric = evaluate.load("seqeval")

In [ ]:
# start from "0"
label_list = list(label_dict.keys())
print("label_dict", label_dict)
print("label_list", label_list)

label_dict {'section': 0, 'other': 1, 'title': 2, 'form_key': 3, 'table_key': 4, 'form_value': 5, 'table_value': 6}
label_list ['section', 'other', 'title', 'form_key', 'table_key', 'form_value', 'table_value']


In [ ]:
# original - matching with target_id, but causing an error

# label_list = list(label_dict.keys())
# print("label_dict", label_dict)
# print("label_list", label_list) #label_dict['Title'] = 1

label_dict {'Title': 1, 'Section': 2, 'Other': 3, 'Form_key': 4, 'Form_value': 5, 'Table_key': 6, 'Table_value': 7}
label_list ['Title', 'Section', 'Other', 'Form_key', 'Form_value', 'Table_key', 'Table_value']


In [ ]:
import numpy as np

#return_entity_level_metrics = True
return_entity_level_metrics = False
label_list = list(label_dict.keys())

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    true_predictions = [
        [label_list[int(p)] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[int(l)] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = metric.compute(predictions=true_predictions, references=true_labels)
    report = classification_report(y_true=true_labels, y_pred=true_predictions, digits=4)

    if return_entity_level_metrics:
        final_results = {}
        for key, value in results.items():
            if isinstance(value, dict):
                for n, v in value.items():
                    final_results[f"{key}_{n}"] = v
            else:
                final_results[key] = value
        return final_results
    else:
        return {
            "overall_precision": results["overall_precision"],
            "overall_recall": results["overall_recall"],
            "overall_f1": results["overall_f1"],
            "overall_accuracy": results["overall_accuracy"],
            "classification_report" : report,
        }

## Training

In [ ]:
#train_dataset = CustomDataset(doc_info=train_all_feat)
train_dataset = CustomDataset(doc_info=train_sample)
val_dataset = CustomDataset(doc_info=val_all_feat)
test_dataset = CustomDataset(doc_info=test_all_feat)

train_dataloader = DataLoader(train_dataset, batch_size=2, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=2, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=2, shuffle=True)

In [ ]:
#put the model in training mode
loss_function = torch.nn.CrossEntropyLoss()
optimizer = AdamW(model.parameters(), lr=2e-5)
model.train()

def train(num_train_epochs):
  #for epoch in range(num_train_epochs):
  for _ in range(num_train_epochs):
      total_loss = 0
      #print("Epoch:", epoch+1)

      for data in tqdm(train_dataloader):
          # get the inputs;
          ids = data['ids'].to(device, dtype = torch.long)
          mask = data['mask'].to(device, dtype = torch.float)
          token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
          labels = data['target'].to(device, dtype = torch.long)
          visual_feats = data['visual_feat'].to(device, dtype = torch.float)
          bert_cls = data['bert_cls'].to(device, dtype = torch.float)
          positional_encoding = data['positional_encoding'].to(device, dtype = torch.float)
          norm_bbox = data['norm_bbox'].to(device, dtype = torch.float)
          object_mask = data['object_mask'].to(device, dtype = torch.float)

          optimizer.zero_grad()
          outputs = model(ids, mask, token_type_ids, bert_cls, visual_feats, positional_encoding, object_mask, norm_bbox)

          # Change the number of categories
          loss = loss_function(outputs.view(-1, 7), labels.view(-1)) # 7 labels
          loss.backward()
          total_loss += loss.item()
          optimizer.step()
      print("Train Loss:", total_loss/len(train_dataloader))


def eval(test_dataloader):
  preds_val = None
  out_label_ids = None
  model.eval()
  total_loss = 0

  for data in tqdm(test_dataloader):
      with torch.no_grad():
          # get the inputs;
          ids = data['ids'].to(device, dtype = torch.long)
          mask = data['mask'].to(device, dtype = torch.float)
          token_type_ids = data['token_type_ids'].to(device, dtype = torch.long)
          labels = data['target'].to(device, dtype = torch.long)
          visual_feats = data['visual_feat'].to(device, dtype = torch.float)
          bert_cls = data['bert_cls'].to(device, dtype = torch.float)
          positional_encoding = data['positional_encoding'].to(device, dtype = torch.float)
          norm_bbox = data['norm_bbox'].to(device, dtype = torch.float)
          object_mask = data['object_mask'].to(device, dtype = torch.float)

          optimizer.zero_grad()
          outputs = model(ids, mask, token_type_ids, bert_cls, visual_feats, positional_encoding, object_mask, norm_bbox)

          loss = loss_function(outputs.view(-1, 7), labels.view(-1)) # 7 labels
          total_loss += loss.item()

          if preds_val is None:
            preds_val = outputs.detach().cpu().numpy()
            out_label_ids = data["target"].detach().cpu().numpy()

          else:
            preds_val = np.append(preds_val, outputs.detach().cpu().numpy(), axis=0)
            out_label_ids = np.append(out_label_ids, data["target"].detach().cpu().numpy(), axis=0)

  print("Val Loss:", total_loss/len(test_dataloader))
  pprint.pprint(compute_metrics((preds_val, out_label_ids)))
  return compute_metrics((preds_val, out_label_ids))

VisualBert - val - 10 ep, 2 batch

    'overall_accuracy': 0.9850374064837906,
    'overall_f1': 0.920585967617579,
    'overall_precision': 0.8937125748502994,
    'overall_recall': 0.9491255961844197}

In [ ]:
current_f1 = 0
best_epoch = 0
optimizer = AdamW(model.parameters(), lr=1e-5)

for epoch in tqdm(range(3)):
  print("Epoch:", epoch+1)
  train(1)
  val_result = eval(val_dataloader)
  if val_result['overall_f1'] > current_f1:
    current_f1 = val_result['overall_f1']
    best_epoch = epoch
    best_val_result = val_result
    torch.save(model.state_dict(), '/content/formnlu_visualbert_best.pth')

  print("Best Epoch:", best_epoch+1)
  pprint.pprint(best_val_result)


  0%|          | 0/3 [00:00<?, ?it/s]

Epoch: 1


  0%|          | 0/268 [00:00<?, ?it/s]

Train Loss: 0.06527214384821953


  0%|          | 0/38 [00:00<?, ?it/s]

...

Val Loss: 0.027778756512194185
{'classification_report': '              precision    recall  f1-score   '
                          'support\n'
                          '\n'
                          '    able_key     0.8659    0.9342    0.8987        '
                          '76\n'
                          '  able_value     0.8922    0.9681    0.9286        '
                          '94\n'
                          '      ection     0.9277    0.9277    0.9277        '
                          '83\n'
                          '        itle     1.0000    1.0000    1.0000        '
                          '76\n'
                          '     orm_key     0.9880    1.0000    0.9939        '
                          '82\n'
                          '   orm_value     0.9815    1.0000    0.9907       '
                          '106\n'
                          '        ther     0.9386    0.9554    0.9469       '
                          '112\n'
                          '\n'
   

  0%|          | 0/268 [00:00<?, ?it/s]

Train Loss: 0.0004436065320797793


  0%|          | 0/38 [00:00<?, ?it/s]

Val Loss: 0.029593729219892628
{'classification_report': '              precision    recall  f1-score   '
                          'support\n'
                          '\n'
                          '    able_key     0.8554    0.9342    0.8931        '
                          '76\n'
                          '  able_value     0.8922    0.9681    0.9286        '
                          '94\n'
                          '      ection     0.9277    0.9277    0.9277        '
                          '83\n'
                          '        itle     1.0000    1.0000    1.0000        '
                          '76\n'
                          '     orm_key     0.9880    1.0000    0.9939        '
                          '82\n'
                          '   orm_value     0.9815    1.0000    0.9907       '
                          '106\n'
                          '        ther     0.9386    0.9554    0.9469       '
                          '112\n'
                          '\n'
   

## Evaluation

LXMERT - 2 epochs

    'overall_accuracy': 0.9971830985915493,
    'overall_f1': 0.9847170590665013,
    'overall_precision': 0.977850697292863,
    'overall_recall': 0.9916805324459235

VisualBert - 2 epochs

    'overall_accuracy': 0.9183098591549296,
    'overall_f1': 0.6864251738016832,
    'overall_precision': 0.6126714565643371,
    'overall_recall': 0.7803660565723793

VisualBert - 10 epochs

    'overall_accuracy': 0.9733477789815818,
    'overall_f1': 0.8655660377358491,
    'overall_precision': 0.8204172876304023,
    'overall_recall': 0.9159733777038269

VisualBert - 20 epochs

    'overall_accuracy': 0.9835319609967498,
    'overall_f1': 0.9186795491143317,
    'overall_precision': 0.890015600624025,
    'overall_recall': 0.9492512479201332

In [ ]:
state_dict = torch.load('/content/formnlu_visualbert_best.pth')
model.load_state_dict(state_dict)
model.eval()
test_result = eval(test_dataloader)

  0%|          | 0/73 [00:00<?, ?it/s]

Val Loss: 0.05847202066777829
{'classification_report': '              precision    recall  f1-score   '
                          'support\n'
                          '\n'
                          '    able_key     0.8025    0.8904    0.8442       '
                          '146\n'
                          '  able_value     0.8621    0.9459    0.9021       '
                          '185\n'
                          '      ection     0.8412    0.8938    0.8667       '
                          '160\n'
                          '        itle     1.0000    1.0000    1.0000       '
                          '146\n'
                          '     orm_key     0.8343    0.9156    0.8731       '
                          '154\n'
                          '   orm_value     0.9556    0.9908    0.9729       '
                          '217\n'
                          '        ther     0.9227    0.9845    0.9526       '
                          '194\n'
                          '\n'
    

In [ ]:
!cp -r /content/formnlu_visualbert_best.pth /content/drive/MyDrive/00_torino_adnlp/formnlu_features/